In [1]:
#Importing modules
import pandas as pd
import numpy as np
import warnings
from datetime import datetime
warnings.filterwarnings('ignore')

In [3]:
#Reading demographic data from local location
demo_init=pd.read_csv(r'D:\Coding\projects\usurious_sisters\Test\test_Data.csv')
print(demo_init.shape)
demo_init.tail()

(14745, 25)


,ID,Frequency,InstlmentMode,LoanStatus,PaymentMode,BranchID,Area,Tenure,AssetCost,AmountFinance,...,AssetID,ManufacturerID,SupplierID,LTV,SEX,AGE,MonthlyIncome,City,State,ZiPCODE
14740,143396,Monthly,Arrear,Closed,PDC,143,NEW DELHI BARAKHAMBHA ROAD,35,530016,419616.0,...,4276747,1046.0,22094,79.17,NaN,NaN,NaN,GURGAON,HARYANA,122001.0
14741,143397,Monthly,Arrear,Closed,PDC,32,BANGALORE LALBAGH,36,595000,446500.0,...,4746586,2721.0,61196,75.04,NaN,NaN,NaN,BANGALORE,KARNATAKA,560020.0
14742,143398,Monthly,Arrear,Closed,PDC,32,BANGALORE LALBAGH,36,595000,446500.0,...,4746588,2721.0,61196,75.04,NaN,NaN,NaN,BANGALORE,KARNATAKA,560020.0
14743,143399,Monthly,Arrear,Closed,PDC_E,246,BIKANER,12,400000,280000.0,...,11719081,1046.0,63565,70.00,M,NaN,NaN,GANGANAGAR,RAJASTHAN,335001.0
14744,143400,Monthly,Arrear,Closed,PDC_E,246,BIKANER,12,400000,280000.0,...,11719080,1046.0,63565,70.00,M,NaN,NaN,GANGANAGAR,RAJASTHAN,335001.0


In [4]:
#printing all column names for reference
print(demo_init.columns)

Index(['ID', 'Frequency', 'InstlmentMode', 'LoanStatus', 'PaymentMode',
       'BranchID', 'Area', 'Tenure', 'AssetCost', 'AmountFinance',
       'DisbursalAmount', 'EMI', 'DisbursalDate', 'MaturityDAte', 'AuthDate',
       'AssetID', 'ManufacturerID', 'SupplierID', 'LTV', 'SEX', 'AGE',
       'MonthlyIncome', 'City', 'State', 'ZiPCODE'],
      dtype='object')


In [5]:
#estimating the fill rates of columns. 
df_nullrate = pd.DataFrame([demo_init.isna().sum()]).transpose()
df_nullrate.rename(columns = {0:'total null rows'}, inplace = True)
df_nullrate['percent_nulls'] = df_nullrate['total null rows']/128655*100
df_nullrate

,total null rows,percent_nulls
ID,0,0.000000
Frequency,0,0.000000
InstlmentMode,0,0.000000
LoanStatus,0,0.000000
PaymentMode,0,0.000000
BranchID,0,0.000000
Area,546,0.424391
Tenure,0,0.000000
AssetCost,0,0.000000
AmountFinance,0,0.000000


In [7]:
#For these categorical variables, we will have one hot vectors - getting their unique values
print(demo_init["Frequency"].unique())
print(demo_init["LoanStatus"].unique())
print(demo_init["InstlmentMode"].unique())
print(demo_init["PaymentMode"].unique())
print(demo_init["State"].unique()) #21 states in total.

['Monthly' 'Quatrly' 'Half Yearly']
['Closed' 'Active']
['Advance' 'Arrear']
['PDC_E' 'PDC' 'Billed' 'Direct Debit' 'ECS' 'ECS Reject' 'SI Reject'
 'Cheque' 'Auto Debit' 'PDC Reject']
['MADHYA PRADESH' 'CHATTISGARH' 'ORISSA' 'WEST BENGAL' 'HARYANA'
 'RAJASTHAN' 'PUNJAB' 'UTTAR PRADESH' 'UTTARAKHAND' 'ANDHRA PRADESH'
 'GUJARAT' 'MAHARASHTRA' 'KARNATAKA' 'TELANGANA' 'JHARKHAND' 'DELHI'
 'BIHAR' 'HIMACHAL PRADESH']


In [8]:
#converting the binary variables first
#if Loan status is closed then 0 if active then 1
demo_init["LOAN_STATUS"] = 1000
demo_init.loc[demo_init["LoanStatus"]=="Closed", "LOAN_STATUS" ] = 0
demo_init.loc[demo_init["LoanStatus"]=="Active", "LOAN_STATUS" ] = 1

In [9]:
#if Instalment Mode is arrear then 0 if advance then 1
demo_init["INS_MODE"] = 1000
demo_init.loc[demo_init["InstlmentMode"]=="Arrear", "INS_MODE" ] = 0
demo_init.loc[demo_init["InstlmentMode"]=="Advance", "INS_MODE" ] = 1

categories for payment mode:
ECS, Auto Debit, Escrow, SI 
Cheque, PDC, PDC_E
Billed, Direct Debit
SI reject, ECS reject, PDC reject

In [10]:
#clubbing payment mode into descriptive categories - automated, cheque, discretionary and reject categories.
#We will be using these new categories for generating one hot vectors.
demo_init["PYMNT_MODE"] = 1000
demo_init.loc[demo_init["PaymentMode"].isin(["ECS", "Direct Debit", "Auto Debit", "Escrow", "SI"]), "PYMNT_MODE"] = "automated"
demo_init.loc[demo_init["PaymentMode"].isin(["Cheque", "PDC", "PDC_E"]), "PYMNT_MODE" ] = "cheque"
demo_init.loc[demo_init["PaymentMode"].isin(["Billed"]), "PYMNT_MODE" ] = "discretionary"
demo_init.loc[demo_init["PaymentMode"].isin(["SI Reject", "ECS Reject", "PDC Reject"]), "PYMNT_MODE" ] = "reject"

In [11]:
#qc: if there is 1000 in unique value of loan Status or Ins_Mode, means either there is a missing value, or the assignment is wrong.
print(demo_init["LOAN_STATUS"].unique())
print(demo_init["INS_MODE"].unique())
print(demo_init["PYMNT_MODE"].unique())
#passed

[0 1]
[1 0]
['cheque' 'discretionary' 'automated' 'reject']


In [12]:
pd.DataFrame(demo_init["Area"].unique()).tail()
#so with 92 values if we create a one hot vector/dummy for each category, the data will become huge. Although, we should see if any of
#these areas have high concentrations of loan topups.

,0
76,PANIPAT
77,KOLKATA WOOD STREET
78,BELAPUR
79,CHURU
80,JALORE


In [13]:
pd.DataFrame(demo_init["City"].unique()).tail()
#so with 489 values if we create a one hot vector/dummy for each category, the data will become huge. Although, we should see if any of
#these areas have high concentrations of loan topups.

,0
316,NALANDA
317,JASHPUR
318,SULTANPUR
319,JALOR
320,PILAKHUWA


In [14]:
#creating column index for demo_init dataframe to merge the one-hot vectors
demo_init.reset_index(inplace = True)

In [15]:
#creating one hot vectors for the remanining, multinomial variabes
one_hot_cols = ["Frequency", "PYMNT_MODE", "State"]
for i in one_hot_cols:
    df_dummy = pd.get_dummies(demo_init[i], prefix='dummy')
    df_dummy.reset_index(inplace=True)
    

    demo_init = pd.merge(demo_init, df_dummy, on = 'index', how = 'left')

In [16]:
print(demo_init.shape)
demo_init.head()

(14745, 54)


,index,ID,Frequency,InstlmentMode,LoanStatus,PaymentMode,BranchID,Area,Tenure,AssetCost,...,dummy_KARNATAKA,dummy_MADHYA PRADESH,dummy_MAHARASHTRA,dummy_ORISSA,dummy_PUNJAB,dummy_RAJASTHAN,dummy_TELANGANA,dummy_UTTAR PRADESH,dummy_UTTARAKHAND,dummy_WEST BENGAL
0,0,4,Monthly,Advance,Closed,PDC_E,2,GUNA,46,480000,...,0,1,0,0,0,0,0,0,0,0
1,1,5,Monthly,Advance,Closed,PDC,2,GUNA,45,480000,...,0,1,0,0,0,0,0,0,0,0
2,2,6,Quatrly,Arrear,Closed,PDC,2,GUNA,48,580000,...,0,1,0,0,0,0,0,0,0,0
3,3,25,Half Yearly,Arrear,Closed,Billed,154,NaN,36,725000,...,0,0,0,0,0,0,0,0,0,0
4,4,119,Quatrly,Arrear,Closed,PDC,194,CUTTACK,48,617000,...,0,0,0,1,0,0,0,0,0,0


In [19]:
#qc - Frequency
demo_init[['Frequency','dummy_Half Yearly','dummy_Monthly','dummy_Quatrly']].head()
#Pass

,Frequency,dummy_Half Yearly,dummy_Monthly,dummy_Quatrly
0,Monthly,0,1,0
1,Monthly,0,1,0
2,Quatrly,0,0,1
3,Half Yearly,1,0,0
4,Quatrly,0,0,1


In [20]:
#qc - PYMNT_MODE
demo_init[['PaymentMode','PYMNT_MODE','dummy_automated','dummy_cheque','dummy_discretionary','dummy_reject']].head()
#pass

,PaymentMode,PYMNT_MODE,dummy_automated,dummy_cheque,dummy_discretionary,dummy_reject
0,PDC_E,cheque,0,1,0,0
1,PDC,cheque,0,1,0,0
2,PDC,cheque,0,1,0,0
3,Billed,discretionary,0,0,1,0
4,PDC,cheque,0,1,0,0


In [23]:
#qc - States

demo_init[['State','dummy_ANDHRA PRADESH',
       'dummy_BIHAR', 'dummy_CHATTISGARH','dummy_DELHI', 'dummy_GUJARAT',
       'dummy_HARYANA', 'dummy_HIMACHAL PRADESH', 'dummy_JHARKHAND',
       'dummy_KARNATAKA', 'dummy_MADHYA PRADESH', 'dummy_MAHARASHTRA',
       'dummy_ORISSA', 'dummy_PUNJAB', 'dummy_RAJASTHAN',
       'dummy_TELANGANA', 'dummy_UTTAR PRADESH', 'dummy_UTTARAKHAND',
       'dummy_WEST BENGAL']].head()
#pass

,State,dummy_ANDHRA PRADESH,dummy_BIHAR,dummy_CHATTISGARH,dummy_DELHI,dummy_GUJARAT,dummy_HARYANA,dummy_HIMACHAL PRADESH,dummy_JHARKHAND,dummy_KARNATAKA,dummy_MADHYA PRADESH,dummy_MAHARASHTRA,dummy_ORISSA,dummy_PUNJAB,dummy_RAJASTHAN,dummy_TELANGANA,dummy_UTTAR PRADESH,dummy_UTTARAKHAND,dummy_WEST BENGAL
0,MADHYA PRADESH,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
1,MADHYA PRADESH,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
2,MADHYA PRADESH,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
3,CHATTISGARH,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,ORISSA,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0


In [27]:
#reading cleaned bureau data
b_df = pd.read_csv(r'D:\Coding\projects\usurious_sisters\Test\bureau_cleandata.csv')
print(b_df.shape)
b_df.head()

(14745, 29)


,ID,Frequency,InstlmentMode,LoanStatus,PaymentMode,BranchID,Area,Tenure,AssetCost,AmountFinance,...,SEX,AGE,MonthlyIncome,City,State,ZiPCODE,ID.1,SUM(DISBURSED-AMT/HIGH CREDIT),SUM(CURRENT-BAL),OTHERS-DISBURSED
0,4,Monthly,Advance,Closed,PDC_E,2,GUNA,46,480000,365000.0,...,M,50.0,32069.00,GUNA,MADHYA PRADESH,473001.0,4,2964588.0,2191274.0,2599588.0
1,5,Monthly,Advance,Closed,PDC,2,GUNA,45,480000,285000.0,...,M,35.0,25000.00,GUNA,MADHYA PRADESH,473001.0,5,1939813.0,907231.0,1654813.0
2,6,Quatrly,Arrear,Closed,PDC,2,GUNA,48,580000,400000.0,...,M,37.0,23333.33,GUNA,MADHYA PRADESH,473001.0,6,3237361.0,102800.0,2837361.0
3,25,Half Yearly,Arrear,Closed,Billed,154,NaN,36,725000,500000.0,...,M,55.0,91666.67,MAHASAMUND,CHATTISGARH,493558.0,25,800000.0,127593.0,300000.0
4,119,Quatrly,Arrear,Closed,PDC,194,CUTTACK,48,617000,400000.0,...,M,48.0,12500.00,JAGATSINGHAPUR,ORISSA,754137.0,119,930000.0,116200.0,530000.0


In [28]:
#merging with bureau data
demo_init = pd.merge(demo_init, b_df[['ID','SUM(DISBURSED-AMT/HIGH CREDIT)','SUM(CURRENT-BAL)','OTHERS-DISBURSED']], on = 'ID', how = 'left')
demo_init.head()

,index,ID,Frequency,InstlmentMode,LoanStatus,PaymentMode,BranchID,Area,Tenure,AssetCost,...,dummy_ORISSA,dummy_PUNJAB,dummy_RAJASTHAN,dummy_TELANGANA,dummy_UTTAR PRADESH,dummy_UTTARAKHAND,dummy_WEST BENGAL,SUM(DISBURSED-AMT/HIGH CREDIT),SUM(CURRENT-BAL),OTHERS-DISBURSED
0,0,4,Monthly,Advance,Closed,PDC_E,2,GUNA,46,480000,...,0,0,0,0,0,0,0,2964588.0,2191274.0,2599588.0
1,1,5,Monthly,Advance,Closed,PDC,2,GUNA,45,480000,...,0,0,0,0,0,0,0,1939813.0,907231.0,1654813.0
2,2,6,Quatrly,Arrear,Closed,PDC,2,GUNA,48,580000,...,0,0,0,0,0,0,0,3237361.0,102800.0,2837361.0
3,3,25,Half Yearly,Arrear,Closed,Billed,154,NaN,36,725000,...,0,0,0,0,0,0,0,800000.0,127593.0,300000.0
4,4,119,Quatrly,Arrear,Closed,PDC,194,CUTTACK,48,617000,...,1,0,0,0,0,0,0,930000.0,116200.0,530000.0


In [29]:
#dropping the index column
demo_init.drop(['index'], axis = 1, inplace = True)
demo_init.head()

,ID,Frequency,InstlmentMode,LoanStatus,PaymentMode,BranchID,Area,Tenure,AssetCost,AmountFinance,...,dummy_ORISSA,dummy_PUNJAB,dummy_RAJASTHAN,dummy_TELANGANA,dummy_UTTAR PRADESH,dummy_UTTARAKHAND,dummy_WEST BENGAL,SUM(DISBURSED-AMT/HIGH CREDIT),SUM(CURRENT-BAL),OTHERS-DISBURSED
0,4,Monthly,Advance,Closed,PDC_E,2,GUNA,46,480000,365000.0,...,0,0,0,0,0,0,0,2964588.0,2191274.0,2599588.0
1,5,Monthly,Advance,Closed,PDC,2,GUNA,45,480000,285000.0,...,0,0,0,0,0,0,0,1939813.0,907231.0,1654813.0
2,6,Quatrly,Arrear,Closed,PDC,2,GUNA,48,580000,400000.0,...,0,0,0,0,0,0,0,3237361.0,102800.0,2837361.0
3,25,Half Yearly,Arrear,Closed,Billed,154,NaN,36,725000,500000.0,...,0,0,0,0,0,0,0,800000.0,127593.0,300000.0
4,119,Quatrly,Arrear,Closed,PDC,194,CUTTACK,48,617000,400000.0,...,1,0,0,0,0,0,0,930000.0,116200.0,530000.0


In [31]:
demo_final = demo_init.drop(columns=["Frequency"
,"InstlmentMode"
,"LoanStatus"
,"PaymentMode"
,"Area"
,"SEX"
,"City"
,"State"
,"PYMNT_MODE"])

In [32]:
#importing the dataset - demo_noqual 
#demo_final = pd.read_csv(r"D:\Coding\projects\usurious_sisters\final_dataset\demo_noqual.csv")
print(demo_final.shape)
demo_final.head()

(14745, 47)


,ID,BranchID,Tenure,AssetCost,AmountFinance,DisbursalAmount,EMI,DisbursalDate,MaturityDAte,AuthDate,...,dummy_ORISSA,dummy_PUNJAB,dummy_RAJASTHAN,dummy_TELANGANA,dummy_UTTAR PRADESH,dummy_UTTARAKHAND,dummy_WEST BENGAL,SUM(DISBURSED-AMT/HIGH CREDIT),SUM(CURRENT-BAL),OTHERS-DISBURSED
0,4,2,46,480000,365000.0,365000.0,1000.0,2011-12-29 00:00:00,2015-10-05 00:00:00,2011-12-29 00:00:00,...,0,0,0,0,0,0,0,2964588.0,2191274.0,2599588.0
1,5,2,45,480000,285000.0,285000.0,9300.0,2012-04-28 00:00:00,2016-01-01 00:00:00,2012-04-28 00:00:00,...,0,0,0,0,0,0,0,1939813.0,907231.0,1654813.0
2,6,2,48,580000,400000.0,400000.0,35800.0,2013-10-22 00:00:00,2017-09-10 00:00:00,2013-10-22 00:00:00,...,0,0,0,0,0,0,0,3237361.0,102800.0,2837361.0
3,25,154,36,725000,500000.0,500000.0,52000.0,2013-05-21 00:00:00,2016-02-10 00:00:00,2013-05-21 00:00:00,...,0,0,0,0,0,0,0,800000.0,127593.0,300000.0
4,119,194,48,617000,400000.0,400000.0,35400.0,2012-11-30 00:00:00,2016-12-05 00:00:00,2012-11-30 00:00:00,...,1,0,0,0,0,0,0,930000.0,116200.0,530000.0


In [33]:
#removing the timestamp from date columns
demo_final['Disbursal_Dt'] = demo_final.DisbursalDate.str[:10]
demo_final['Maturity_Dt'] = demo_final.MaturityDAte.str[:10]
demo_final['Auth_Dt'] = demo_final.AuthDate.str[:10]
demo_final[['Disbursal_Dt','Maturity_Dt','Auth_Dt']].head()

,Disbursal_Dt,Maturity_Dt,Auth_Dt
0,2011-12-29,2015-10-05,2011-12-29
1,2012-04-28,2016-01-01,2012-04-28
2,2013-10-22,2017-09-10,2013-10-22
3,2013-05-21,2016-02-10,2013-05-21
4,2012-11-30,2016-12-05,2012-11-30


In [34]:
#dropping the old Date columns
demo_final.drop(['DisbursalDate','MaturityDAte', 'AuthDate'], inplace = True, axis = 1)
print(demo_final.shape)

(14745, 47)


In [36]:
#creating unix timestamps for dates
demo_final['Disbursal_Dt_U'] = demo_final.Disbursal_Dt.apply(lambda x: (datetime.strptime(x,'%Y-%m-%d')).timestamp())
demo_final['Maturity_Dt_U'] = demo_final.Maturity_Dt.apply(lambda x: (datetime.strptime(x,'%Y-%m-%d')).timestamp())
demo_final['Auth_Dt_U'] = demo_final.Auth_Dt.apply(lambda x: (datetime.strptime(x,'%Y-%m-%d')).timestamp())

In [37]:
#converting dates from string to datetime objects
demo_final['Disbursal_Dt'] = demo_final.Disbursal_Dt.apply(lambda x: datetime.strptime(x,'%Y-%m-%d'))
demo_final['Maturity_Dt'] = demo_final.Maturity_Dt.apply(lambda x: datetime.strptime(x,'%Y-%m-%d'))
demo_final['Auth_Dt'] = demo_final.Auth_Dt.apply(lambda x: datetime.strptime(x,'%Y-%m-%d'))

In [38]:
#QC
demo_final[['Maturity_Dt','Disbursal_Dt','Auth_Dt']].head()

,Maturity_Dt,Disbursal_Dt,Auth_Dt
0,2015-10-05,2011-12-29,2011-12-29
1,2016-01-01,2012-04-28,2012-04-28
2,2017-09-10,2013-10-22,2013-10-22
3,2016-02-10,2013-05-21,2013-05-21
4,2016-12-05,2012-11-30,2012-11-30


In [41]:
#adding these dummies, setting these values to 0 since we have them in train data too
demo_final['dummy_DADRA AND NAGAR HAVELI']=0
demo_final['dummy_BI-Monthly'] =0
demo_final['dummy_CHANDIGARH'] =0
demo_final['dummy_TAMIL NADU']=0
demo_final['dummy_ASSAM']=0

In [42]:
print(demo_final.shape)
demo_final.head()

(14745, 55)


,ID,BranchID,Tenure,AssetCost,AmountFinance,DisbursalAmount,EMI,AssetID,ManufacturerID,SupplierID,...,Maturity_Dt,Auth_Dt,Disbursal_Dt_U,Maturity_Dt_U,Auth_Dt_U,dummy_DADRA AND NAGAR HAVELI,dummy_BI-Monthly,dummy_CHANDIGARH,dummy_TAMIL NADU,dummy_ASSAM
0,4,2,46,480000,365000.0,365000.0,1000.0,3524747,1046.0,22354,...,2015-10-05,2011-12-29,1.325097e+09,1.443983e+09,1.325097e+09,0,0,0,0,0
1,5,2,45,480000,285000.0,285000.0,9300.0,4985862,1046.0,22354,...,2016-01-01,2012-04-28,1.335551e+09,1.451587e+09,1.335551e+09,0,0,0,0,0
2,6,2,48,580000,400000.0,400000.0,35800.0,12881783,1060.0,65929,...,2017-09-10,2013-10-22,1.382380e+09,1.504982e+09,1.382380e+09,0,0,0,0,0
3,25,154,36,725000,500000.0,500000.0,52000.0,10613776,1049.0,68597,...,2016-02-10,2013-05-21,1.369075e+09,1.455043e+09,1.369075e+09,0,0,0,0,0
4,119,194,48,617000,400000.0,400000.0,35400.0,7911110,1049.0,38493,...,2016-12-05,2012-11-30,1.354214e+09,1.480876e+09,1.354214e+09,0,0,0,0,0


In [43]:
#saving the final dataset - with normal dates
demo_final.to_csv(r"D:\Coding\projects\usurious_sisters\final_dataset\final_data_test.csv", index = False)